# PIT according to Tongtong

# Purpose
Implement PIT accoring to:

[1]
Wang, Tongtong, Guoyuan Li, Baiheng Wu, Vilmar Æsøy, and Houxiang Zhang. “Parameter Identification of Ship Manoeuvring Model Under Disturbance Using Support Vector Machine Method.” Ships and Offshore Structures, May 19, 2021.


# Methodology
* [1] has a an interesing way to do the regression ([1]eq.3.). This will be tested on the present data.

# Setup

In [ ]:
# %load imports.py
## Local packages:

%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import numpy as np
import os
import matplotlib.pyplot as plt
#if os.name == 'nt':
#    plt.style.use('presentation.mplstyle')  # Windows

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify

import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp

## Local packages:
from vessel_manoeuvring_models.data import mdl

# Load models:
# (Uncomment these for faster loading):
import vessel_manoeuvring_models.models.vmm_abkowitz  as vmm_abkowitz  

from vessel_manoeuvring_models.symbols import *
from vessel_manoeuvring_models.parameters import *
import vessel_manoeuvring_models.symbols as symbols
from vessel_manoeuvring_models import prime_system
from vessel_manoeuvring_models.models import regression
from vessel_manoeuvring_models.visualization.plot import track_plot
from vessel_manoeuvring_models.equation import Equation

## Load test

In [ ]:
#id=22773
#id=22616
id=22774
#id=22770


df, units, meta_data = mdl.load(id=id, dir_path='../data/processed/kalman_cut/')
df.index = df.index.total_seconds()
df.index-=df.index[0]

df['t'] = df.index
df.sort_index(inplace=True)
df['-delta'] = -df['delta']
df['V'] = np.sqrt(df['u']**2 + df['v']**2)

df['thrust'] = df['Prop/PS/Thrust'] + df['Prop/SB/Thrust']
df['U'] = df['V']
df['beta'] = -np.arctan2(df['v'],df['u'])

In [ ]:
meta_data['rho']=1000
meta_data['mass'] = meta_data['Volume']*meta_data['rho']

In [ ]:
from vessel_manoeuvring_models.visualization.plot import track_plot
fig,ax=plt.subplots()
#fig.set_size_inches(10,10)
track_plot(df=df, lpp=meta_data.lpp, x_dataset='x0', y_dataset='y0',  psi_dataset='psi', beam=meta_data.beam, ax=ax);
df.plot(y='u')

# Ship parameters

In [ ]:
T_ = (meta_data.TA + meta_data.TF)/2
L_ = meta_data.lpp
m_ = meta_data.mass
rho_ = meta_data.rho
B_ = meta_data.beam
CB_ = m_/(T_*B_*L_*rho_)
I_z_ = m_*meta_data.KZZ**2
#I_z_=839.725

ship_parameters = {
        'T' : T_,
        'L' : L_,
        'CB' :CB_,
        'B' : B_,
        'rho' : rho_,
        #'x_G' : meta_data.lcg,  # motions are expressed at CG
        'x_G' : 0,  # motions are expressed at CG
        
        'm' : m_,
        'I_z': I_z_, 
        'volume':meta_data.Volume,
    }

ps = prime_system.PrimeSystem(**ship_parameters)  # model

scale_factor = meta_data.scale_factor
ps_ship = prime_system.PrimeSystem(L=ship_parameters['L']*scale_factor, rho=meta_data['rho'])  # ship
ship_parameters_prime = ps.prime(ship_parameters)

## Prime system

In [ ]:
interesting = ['x0','y0','psi','u','v','r','u1d','v1d','r1d','U','t','delta','thrust','beta']
df_prime = ps.prime(df[interesting], U=df['U'])
df_prime.set_index('t', inplace=True)

In [ ]:
fig,ax=plt.subplots()
ax.plot(df.index, df_prime.index)

In [ ]:
fig,ax=plt.subplots()
#fig.set_size_inches(10,10)
track_plot(df=df_prime, lpp=ship_parameters_prime['L'], beam=ship_parameters_prime['B'],
           x_dataset='x0', y_dataset='y0',  psi_dataset='psi', ax=ax);

df_prime.plot(y='u')

# Brix parameters

In [ ]:
def calculate_prime(row, ship_parameters):
    return run(function=row['brix_lambda'], inputs=ship_parameters)


mask = df_parameters['brix_lambda'].notnull()
df_parameters.loc[mask,'brix_prime'] = df_parameters.loc[mask].apply(calculate_prime, ship_parameters=ship_parameters, axis=1)

df_parameters.loc['Ydelta','brix_prime'] = 0.0004  # Just guessing
df_parameters.loc['Ndelta','brix_prime'] = -df_parameters.loc['Ydelta','brix_prime']/4  # Just guessing

df_parameters['brix_prime'].fillna(0, inplace=True)
#df_parameters['brix_SI'].fillna(0, inplace=True)

df_parameters.groupby(by='state').get_group('dot')

## Regression model
$$
\begin{array}{lcl}
u'(n+1) - u'(n) = A \cdot X(n) \\
v'(n+1) - v'(n) = B \cdot Y(n) \\
r'(n+1) - r'(n) = C \cdot N(n) \\
\end{array}
$$
where A, B, C are parameter vectors formed by hydrodynamic
derivatives to be identified.

In [ ]:
label = sp.symbols('X_qs')
X_eq_ = vmm_abkowitz.X_qs_eq.subs(vmm_abkowitz.X_qs,label)
diff_eq_X = regression.DiffEqToMatrix(ode=X_eq_, label=label, base_features=[delta,u,v,r,thrust])

label = sp.symbols('Y_qs')
Y_eq_ = vmm_abkowitz.Y_qs_eq.subs(vmm_abkowitz.Y_qs,label)
diff_eq_Y = regression.DiffEqToMatrix(ode=Y_eq_, label=label, base_features=[delta,u,v,r])

label = sp.symbols('N_qs')
N_eq_ = vmm_abkowitz.N_qs_eq.subs(vmm_abkowitz.N_qs,label)
diff_eq_N = regression.DiffEqToMatrix(ode=N_eq_, label=label, base_features=[delta,u,v,r])




In [ ]:
Math(vlatex(diff_eq_X.acceleration_equation))

In [ ]:
Math(vlatex(diff_eq_X.eq_X))

In [ ]:
X_martin = diff_eq_X.eq_X.rhs
X_tongtong = X_martin*U**2/L*delta_t/(m-p.Xudot)

In [ ]:
S = (m-p.Yvdot)*(I_z-p.Nrdot) - (m*x_G-p.Yrdot)*(m*x_G-p.Nvdot)

In [ ]:
Y_martin = diff_eq_Y.eq_X.rhs
Y_tongtong = Y_martin*U**2/L*delta_t/(S)

N_martin = diff_eq_N.eq_X.rhs
N_tongtong = N_martin*U**2/L**2*delta_t/(S)

In [ ]:
subs = {value:key for key,value in p.items()}
X_lambda=lambdify(X_tongtong.subs(subs))
Y_lambda=lambdify(Y_tongtong.subs(subs))
N_lambda=lambdify(N_tongtong.subs(subs))

In [ ]:
from statsmodels.sandbox.regression.predstd import wls_prediction_std
def show_pred(X,y,results, label):
    
    display(results.summary())
    
    X_ = X
    y_ = y
    y_pred = results.predict(X_)
    
    prstd, iv_l, iv_u = wls_prediction_std(results, exog=X_, alpha=0.05)
    #iv_l*=-1 
    #iv_u*=-1
    
    fig,ax=plt.subplots()
    ax.plot(X_.index,y_, label='Numerical gradient from model test')
    ax.plot(X_.index,y_pred, '--', label='OLS')
    ax.set_ylabel(label)
    
    ax.fill_between(X_.index, y1=iv_l, y2=iv_u, zorder=-10, color='grey', alpha=0.5, label=r'5\% confidence')
    ax.legend();

In [ ]:
def regress(data, lamda, diff_eq, y_key, U_):
    
    X = run(lamda, inputs=data,U=U_,
            L=ship_parameters['L'], m=ship_parameters_prime['m'],
            x_G=ship_parameters_prime['x_G'], I_z=ship_parameters_prime['I_z'],
            **df_parameters['brix_prime'], delta_t=dt_)[0].T

    subs = diff_eq.feature_names_subs()        
    columns = list(subs.values())
    X = pd.DataFrame(data=X, index=data.index, columns=columns)
    y = df_prime[y_key].diff().iloc[1:].copy()
    
    model = sm.OLS(y,X)
    results = model.fit()
    
    label=f"{y_key}'(n+1) - {y_key}'(n)"
    show_pred(X=X,y=y,results=results, label=label)
    
    return model,results

In [ ]:
#dt_ = np.diff(df_prime.index)  # SI or prime?
dt_ = np.mean(np.diff(df.index))  # SI or prime?
data = df_prime.iloc[1:].copy()
data.drop(columns='U',inplace=True)
U_ = df.iloc[1:]['U'].copy().values

## X

In [ ]:
model_X, results_X = regress(data=data, lamda=X_lambda, diff_eq=diff_eq_X, y_key='u', U_=U_)

## Y

In [ ]:
model_Y, results_Y = regress(data=data, lamda=Y_lambda, diff_eq=diff_eq_Y, y_key='v',  U_=U_)

## N

In [ ]:
model_N, results_N = regress(data=data, lamda=N_lambda, diff_eq=diff_eq_N, y_key='r',  U_=U_)

In [ ]:
results_summary_X = regression.results_summary_to_dataframe(results_X)
results_summary_Y = regression.results_summary_to_dataframe(results_Y)
results_summary_N = regression.results_summary_to_dataframe(results_N)

In [ ]:
A_ = results_summary_X['coeff']
B_ = results_summary_Y['coeff']
C_ = results_summary_N['coeff']

In [ ]:
eq_X_coeff = sp.Eq(X_coeff,L*(m-p.Xudot)/delta_t*A_coeff)
eq_X_coeff

In [ ]:
subs = {value:key for key,value in p.items()}
solution_X_lambda = lambdify(eq_X_coeff.rhs.subs(subs))

In [ ]:
M = sp.matrices.MutableDenseMatrix([
                                [(I_z-p.Nrdot)*delta_t/(S*L), -(m*x_G-p.Yrdot)*delta_t/(S*L)],
                                [-(m*x_G-p.Nvdot)*delta_t/(S*L**2), (m-p.Yvdot)*delta_t/(S*L**2)]
                               ])

In [ ]:
solution = M.inv()*sp.matrices.MutableDenseMatrix([B_coeff,C_coeff])
subs = {value:key for key,value in p.items()}
solution_YN_lambda = lambdify(solution.subs(subs))

In [ ]:
parameters_A = run(solution_X_lambda, A_coeff=A_, delta_t=dt_, **df_parameters['brix_prime'],
                  L=ship_parameters['L'], m=ship_parameters_prime['m'])

In [ ]:
parameters_ = run(solution_YN_lambda, B_coeff=B_.values, C_coeff=C_.values, delta_t=dt_, 
                  L=ship_parameters['L'], m=ship_parameters_prime['m'], x_G=ship_parameters_prime['x_G'],
                  I_z=ship_parameters_prime['I_z'],
                  **df_parameters['brix_prime'])

In [ ]:
parameters_[0][0]

In [ ]:
parameters_B = pd.Series(parameters_[0][0], index=B_.index)
parameters_C = pd.Series(parameters_[1][0], index=C_.index)

df_parameters['regressed'] = parameters_A
df_parameters['regressed'] = df_parameters['regressed'].combine_first(parameters_B)
df_parameters['regressed'] = df_parameters['regressed'].combine_first(parameters_C)

df_parameters['prime'] = df_parameters['regressed'].combine_first(df_parameters['brix_prime'])

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(15,5)

mask = ((df_parameters['brix_prime']!=0) |
        (pd.notnull(df_parameters['regressed']))
       )
                
df_parameters_all_plot = df_parameters.loc[mask]

df_parameters_all_plot.drop(index=['Xthrust']).plot.bar(y=['brix_prime','regressed'], ax=ax);

In [ ]:
df_prime_ = df_prime.copy()
df_prime_.index = df.index
result = vmm_abkowitz.simulator.simulate(df_=df_prime_, parameters=df_parameters['prime'], 
                                ship_parameters=ship_parameters_prime)

In [ ]:
result.plot_compare()